In [1]:
%cd ..

c:\Code\Motor-Execution-Classifiaction


In [1]:
!pip install playsound

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for playsound: started
  Running setup.py install for playsound: finished with status 'done'


  DEPRECATION: playsound is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [2]:
import pandas as pd
import numpy as np

import os

import mne
import numpy as np
import pandas as pd

from config.arms_offline import config

%matplotlib qt

In [4]:
from playsound import playsound

In [8]:
!pip uninstall playsound


: 

: 

In [ ]:
!pip install playsound==1.2.2

In [7]:
playsound('/resources/Voices/fel_male.mp3')
print('playing sound using  playsound')


    Error 275 for command:
        open /resources/Voices/fel_male.mp3
    A megadott fájl nem található. Győződjön meg az elérési út és a fájlnév helyességéről.

    Error 263 for command:
        close /resources/Voices/fel_male.mp3
    A megadott eszköz nincs megnyitva, illetve az MCI nem ismeri fel.
Failed to close the file: /resources/Voices/fel_male.mp3


PlaysoundException: 
    Error 275 for command:
        open /resources/Voices/fel_male.mp3
    A megadott fájl nem található. Győződjön meg az elérési út és a fájlnév helyességéről.

In [3]:
subject = 'Viktor'
session = 'session_01'

In [4]:
def loadData(subject, session, tmin=5):
    """
    Load the numpy session
    and convert it to mne dataset
    """
    path = os.path.join(config.data_path, subject, session, "data.npy")
    test = np.load(path)
    EEG = test[2:]
    EEG.shape
    EEG /= 1000 * 1000  # (µV->V) #for Unicorn
    test[2:] = EEG

    sfreq = 250

    ch_names = [
        "STIM",
        "Fz",
        "C3",
        "Cz",
        "C4",
        "Pz",
        "PO7",
        "Oz",
        "PO8",
    ]  # documentation
    #ch_names = ["STIM", "Fp1", "FC1", "FC2", "FCz", "CPz", "CP1", "CP2", "Pz"] # API
    #ch_names = ["STIM","T7","C5", "C3", "C1", "C2","C4","C6","T8"]

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

    raw = mne.io.RawArray(test[1:10], info)  # Drop timestamp channel

    raw.crop(tmin=tmin)

    for ch in info.ch_names:
        if ch == "STIM":
            raw.set_channel_types({ch: "stim"})
        else:
            raw.set_channel_types({ch: "eeg"})
    raw.set_montage("standard_1020")

    raw.describe()

    return raw


def filter(data):
    if config.bandpass:
        data = data.filter(
            config.bandpass_freq[0], config.bandpass_freq[1], fir_design="firwin"
        )
    if config.notch:
        data = data.notch_filter(config.notch_freq)


def create_epochs(data):
    tmin, tmax = -1, 4
    event_dict = {"Rest": 1, "Left": 2, "Right": 3}  # ,'EndOfEvent': 10}
    mapping = {1: "Rest", 2: "Left", 3: "Right"}  # , 10:'EndOfEvent'}
    events = mne.find_events(data, stim_channel="STIM")
    annot_from_events = mne.annotations_from_events(
        events=events, event_desc=mapping, sfreq=data.info["sfreq"]
    )
    data.set_annotations(annot_from_events)
    data.drop_channels(["STIM"])
    return mne.Epochs(
        data, events,  event_id=event_dict, tmin = tmin - 0.5, tmax = tmax + 0.5, preload=True
    )


In [5]:
raw_data = loadData(subject, session, config.relax_time)
raw_data.describe()

Creating RawArray with float64 data, n_channels=9, n_times=55575
    Range : 0 ... 55574 =      0.000 ...   222.296 secs
Ready.


C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) Fz has changed from NA to V.
  raw.set_channel_types({ch: "eeg"})
C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) C3 has changed from NA to V.
  raw.set_channel_types({ch: "eeg"})
C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) Cz has changed from NA to V.
  raw.set_channel_types({ch: "eeg"})
C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) C4 has changed from NA to V.
  raw.set_channel_types({ch: "eeg"})
C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) Pz has changed from NA to V.
  raw.set_channel_types({ch: "eeg"})
C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:39: RuntimeWarning: The unit for channel(s) PO7 has changed from N

<RawArray | 9 x 53075 (212.3 s), ~3.7 MB, data loaded>
ch  name  type  unit        min         Q1     median         Q3        max
 0  STIM  STIM  NA         0.00       0.00       0.00       0.00      10.00
 1  Fz    EEG   µV    184507.22  184879.61  185071.64  185327.89  186748.30
 2  C3    EEG   µV    174856.00  175403.89  175592.09  175815.56  176406.41
 3  Cz    EEG   µV    243629.56  252282.32  264011.12  283858.36  369878.28
 4  C4    EEG   µV    250289.66  258677.11  269456.28  286778.44  349281.69
 5  Pz    EEG   µV    182548.50  184775.04  186031.16  187919.48  195288.91
 6  PO7   EEG   µV    183387.84  184402.66  185406.56  186175.16  186837.25
 7  Oz    EEG   µV    180728.97  181584.16  182018.41  182473.80  183998.94
 8  PO8   EEG   µV    183675.83  184283.80  184534.77  184766.95  185179.03
<RawArray | 9 x 53075 (212.3 s), ~3.7 MB, data loaded>
ch  name  type  unit        min         Q1     median         Q3        max
 0  STIM  STIM  NA         0.00       0.00       0.00 

In [6]:
filtered_data = raw_data.copy()
filter(filtered_data)
filtered_data.describe()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 8251 samples (33.004 sec)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwid

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


In [7]:
# raw_motor_data = raw_data.copy()
# raw_motor_data.drop_channels(["Fz", "Pz", "PO7", "Oz", "PO8"])


In [8]:
filtered_motor_data = filtered_data.copy()
filtered_motor_data.drop_channels(["Fz", "Pz", "PO7", "Oz", "PO8"])

<RawArray | 4 x 53075 (212.3 s), ~1.6 MB, data loaded>

In [9]:
# epochs = create_epochs(raw_data)

In [10]:
projection = True
raw_avg_ref = filtered_motor_data.copy().set_eeg_reference(ref_channels='average', projection = projection)


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


In [11]:
epochs = create_epochs(raw_avg_ref)



60 events found
Event IDs: [ 1  2  3 10]
Not setting metadata
30 matching events found
Setting baseline interval to [-1.5, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 30 events and 1501 original time points ...
0 bad epochs dropped


C:\Users\Nemes\AppData\Local\Temp\ipykernel_11604\2493959640.py:64: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  data.set_annotations(annot_from_events)


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import pandas as pd
import seaborn as sns
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test

In [13]:
from mne.decoding import Scaler
scaler = Scaler(mne.create_info(ch_names= [
        "STIM",
        "Fz",
        "C3",
        "Cz",
        "C4",
        "Pz",
        "PO7",
        "Oz",
        "PO8",
    ] , sfreq=250))

In [14]:
#epochs = scaler.fit_transform(epochs._data, epochs._)

In [15]:
freqs = np.arange(2, 36)  # frequencies from 2-35Hz
# vmin, vmax = -0.1, 1.5  # set min and max ERDS values in plot
# baseline = (0., 0.1)  # baseline interval (in s)
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
baseline = (-1, 0)  # baseline interval (in s)
cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # min, center & max ERDS

kwargs = dict(n_permutations=10, step_down_p=0.05, seed=1,
              buffer_size=None, out_type='mask')  # for cluster test

In [16]:
# tmin = 0.4
# tmax = 1.5
tmin, tmax = -1, 4
# event_ids = dict(Rest = 1, Left = 2, Right = 3)  # map event IDs to tasks
event_ids = {"Rest": 1, "Left": 2, "Right": 3} 

In [17]:
tfr = tfr_multitaper(epochs, freqs=freqs, n_cycles=freqs, use_fft=True,
                     return_itc=False, average=False, decim=2)
tfr.crop(tmin, tmax).apply_baseline(baseline, mode="percent")

for event in event_ids:
    # select desired epochs for visualization
    tfr_ev = tfr[event]
    fig, axes = plt.subplots(1, 4, figsize=(12, 4),
                             gridspec_kw={"width_ratios": [10, 10, 10, 1]})
    for ch, ax in enumerate(axes[:-1]):  # for each channel
        # positive clusters
        _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch], tail=1, **kwargs)
        # negative clusters
        _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch], tail=-1, **kwargs)

        # note that we keep clusters with p <= 0.05 from the combined clusters
        # of two independent tests; in this example, we do not correct for
        # these two comparisons
        c = np.stack(c1 + c2, axis=2)  # combined clusters
        p = np.concatenate((p1, p2))  # combined p-values
        mask = c[..., p <= 0.05].any(axis=-1)

        # plot TFR (ERDS map with masking)
        tfr_ev.average().plot([ch], cmap="RdBu", cnorm=cnorm, axes=ax,
                              colorbar=False, show=False, mask=mask,
                              mask_style="mask")

        ax.set_title(epochs.ch_names[ch], fontsize=10)
        ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
        if ch != 0:
            ax.set_ylabel("")
            ax.set_yticklabels("")
    fig.colorbar(axes[0].images[-1], cax=axes[-1]).ax.set_yscale("linear")
    fig.suptitle(f"ERDS ({event})")
    plt.show()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
Applying baseline correction (mode: percent)


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s finished


Using a threshold of 1.833113
stat_fun(H1): min=-6.605579 max=4.236734
Running initial clustering …
Found 83 clusters


C:\Users\Nemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| Permuting : 9/9 [00:00<00:00,   82.29it/s]

...

100%|██████████| Permuting : 9/9 [00:00<00:00,  163.56it/s]

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Using a threshold of -1.833113
stat_fun(H1): min=-9.652371 max=4.296559


Running initial clustering …
Found 132 clusters


100%|██████████| Permuting : 9/9 [00:00<00:00,  176.19it/s]

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
No baseline correction applied


In [18]:
df = tfr.to_data_frame(time_format=None, long_format=True)



Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...


In [19]:

# Map to frequency bands:
freq_bounds = {'_': 0,
               'delta': 3,
               'theta': 7,
               'alpha': 13,
               'beta': 35,
               'gamma': 140}
df['band'] = pd.cut(df['freq'], list(freq_bounds.values()),
                    labels=list(freq_bounds)[1:])

# Filter to retain only relevant frequency bands:
freq_bands_of_interest = ['delta', 'theta', 'alpha', 'beta']
df = df[df.band.isin(freq_bands_of_interest)]
df['band'] = df['band'].cat.remove_unused_categories()

# Order channels for plotting:
df['channel'] = df['channel'].cat.reorder_categories(('C3', 'Cz', 'C4'),
                                                     ordered=True)

g = sns.FacetGrid(df, row='band', col='channel', margin_titles=True)
g.map(sns.lineplot, 'time', 'value', 'condition', n_boot=10)
axline_kw = dict(color='black', linestyle='dashed', linewidth=0.5, alpha=0.5)
g.map(plt.axhline, y=0, **axline_kw)
g.map(plt.axvline, x=0, **axline_kw)
g.set(ylim=(None, 1.5))
g.set_axis_labels("Time (s)", "ERDS (%)")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.add_legend(ncol=2, loc='lower center')
g.fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.08)